In [1]:
import os
os.chdir("drive/MyDrive/fyp/src")

In [2]:
from data import PolygonSentenceReader, Polygon
from modules import NLLLoss, ELBOLoss
from neural_process.models.np import CNP, LNP
from neural_process.models.attnnp import AttnCNP, AttnLNP
from utils import plot_polygon

import datetime
import random
from tqdm import tqdm
import torch
import torch.nn as nn
from torch.distributions.kl import kl_divergence

In [3]:
TRAINING_ITERATIONS = int(2e5)
PLOT_AFTER = int(1e4)
BATCH_SIZE = 64
MAX_CONTEXT_POINTS = 10
MAX_SEQ_LEN = 5 + 4 * 8
torch.manual_seed(0)

criterion = ELBOLoss()

# Instantiate a polygon generator.
# (For example, polygons with between 3 and 8 sides.)
polygon_generator_train = PolygonSentenceReader(
    batch_size=BATCH_SIZE,
    max_num_context=MAX_CONTEXT_POINTS,
    max_seq_len=MAX_SEQ_LEN,
    min_num_sides=3,
    max_num_sides=8,
    center=(5, 5),
    radius=3,
    testing=False,
)

polygon_generator_test = PolygonSentenceReader(
    batch_size=100,
    max_num_context=MAX_CONTEXT_POINTS,
    max_seq_len=MAX_SEQ_LEN,
    min_num_sides=4,
    max_num_sides=4,
    center=(5, 5),
    radius=3,
    testing=True,
)

In [4]:
model = LNP(x_dim=MAX_SEQ_LEN, y_dim=MAX_SEQ_LEN)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
    optimizer, T_max=TRAINING_ITERATIONS, eta_min=1e-6
)
model.to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))
device = next(model.parameters()).device

# ----------------------
# Training Loop
# ----------------------

progress_bar = tqdm(range(TRAINING_ITERATIONS + 1), desc="Training", unit="iter")
for it in progress_bar:
    # Generate a batch of polygon completion tasks.
    (
        context_x,
        context_y,
        target_x,
        target_y,
        tokens,
        true_poly,
        max_seq_len,
        num_context,
        context_masks,
    ) = polygon_generator_train.generate_polygon_batch_few_shot_masked_completion_task()

    context_x = context_x.to(device)
    context_y = context_y.to(device)
    target_x = target_x.to(device)
    target_y = target_y.to(device)
    context_masks = context_masks.to(device)

    optimizer.zero_grad()
    # Forward pass: the NP model expects context_x, context_y, target_x, target_y.
    dist, z, q_zc, q_zct = model(context_x, context_y, target_x, target_y)
    loss = criterion(dist, q_zct, q_zc, target_y, mask=context_masks)
    loss.backward()

    # torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
    optimizer.step()
    scheduler.step()

    if it % 1000 == 0:
        now = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        progress_bar.set_postfix({"Iteration": it, "Train Loss": f"{loss.item():.4f}", "LR": f"{scheduler.get_last_lr()[0]:.6f}"})
        print(f"{now}: Iteration: {it}, Train Loss: {loss.item():.4f}")

    # ----------------------
    # Evaluation and Plotting
    # ----------------------
    if it % PLOT_AFTER == 0:
        # For plotting, we generate a single polygon sample.
        (
            context_x_eval,
            context_y_eval,
            target_x_eval,
            target_y_eval,
            tokens_eval,
            true_poly_eval,
            max_seq_len_eval,
            num_context_eval,
            context_masks_eval,
        ) = polygon_generator_test.generate_polygon_batch_few_shot_masked_completion_task()

        context_x_eval = context_x_eval.to(device)
        context_y_eval = context_y_eval.to(device)
        target_x_eval = target_x_eval.to(device)
        target_y_eval = target_y_eval.to(device)
        context_masks_eval = context_masks_eval.to(device)

        # Forward pass through the model.
        test_dist, test_z, test_q_zc, test_q_zct = model(
            context_x_eval, context_y_eval, target_x_eval, target_y_eval
        )
        loss_eval = criterion(dist, q_zct, q_zc, target_y, mask=context_masks)

        # Get NP predictions on target tokens (taking the mean of the predictive distribution).
        pred_target = test_dist.mean  # shape: [n_z, batch, n_target, y_dim]

        # For simplicity, we average over latent samples and batch dimension.
        pred_target_avg = pred_target.mean(0)[0]  # shape: [n_target, y_dim]
        # print("pred_target_avg shape: ", pred_target_avg.shape)
        pred_target_avg = pred_target_avg[0]
        true_polygon = true_poly_eval[0]
        true_polygon_tokenised = true_polygon.to_tokenised()
        pred_target_avg = pred_target_avg[: len(true_polygon_tokenised)].tolist()
        # print("pred_target_avg shape after squeeze: ", pred_target_avg.shape)

        # Squeeze out the batch‐and‐context dims to get a 1D mask
        mask_1d = context_masks_eval.squeeze(1)[0].cpu().tolist()
        mask_1d = mask_1d[: len(true_polygon_tokenised)]

        # Construct predicted tokenised form
        predicted_tokens = []
        for t_true, t_pred, m in zip(
            true_polygon_tokenised, pred_target_avg, mask_1d
        ):
            # m == 1: masked token
            predicted_tokens.append(t_pred if m == 1 else t_true)

        # Reconstruct polygons from tokens.
        try:
            predicted_polygon = Polygon.from_tokenised(predicted_tokens, int(true_polygon_tokenised[0]))
        except Exception as e:
            print("Error reconstructing polygon:", e)
            predicted_polygon = None

        now = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        print(f"{now}: Iteration: {it}, Test Loss: {loss_eval.item():.4f}")
        print("True Polygon: ", true_polygon)
        if predicted_polygon is not None:
            print("Predicted Polygon Vertices: ", predicted_polygon)
        else:
            print("Predicted Polygon could not be reconstructed.")

        # Plot true and predicted polygons.
        # plot_polygon(true_polygon.vertices, title="True Polygon")
        # if predicted_polygon is not None:
        #     try:
        #         plot_polygon(predicted_polygon.vertices, title="Predicted Polygon")
        #     except Exception as e:
        #         print("Invalid polygon for plotting:", e)

Training:   0%|          | 4/200001 [00:01<11:58:39,  4.64iter/s, Iteration=0, Train Loss=765430720.0000, LR=0.000100]

2025-04-26 15:47:31: Iteration: 0, Train Loss: 765430720.0000
2025-04-26 15:47:31: Iteration: 0, Test Loss: 765430720.0000
True Polygon:  Polygon(n=4, vertices=[(7.89, 5.81), (3.54, 7.62), (2.03, 5.44), (4.77, 2.01)], lengths=[4.711539026687564, 2.651886121235224, 4.390045557850169, 4.916746892000848], angles=[73.20400494066425, 102.11701644627829, 106.67213898839545, 78.00683962466204])
Predicted Polygon Vertices:  Polygon(n=4, vertices=[(7.89, 5.81), (3.54, 7.62), (2.03, 5.44), (4.77, 2.01)], lengths=[4.711539026687564, 2.651886121235224, 4.390045557850169, 4.916746892000848], angles=[12.446619033813477, -16.73868179321289, 13.744227409362793, 3.5979905128479004])


Training:   1%|          | 1007/200001 [00:34<1:50:08, 30.11iter/s, Iteration=1000, Train Loss=2118067.0000, LR=0.000100]

2025-04-26 15:48:05: Iteration: 1000, Train Loss: 2118067.0000


Training:   1%|          | 2005/200001 [01:08<1:49:54, 30.03iter/s, Iteration=2000, Train Loss=532405.7500, LR=0.000100]

...

2025-04-26 17:37:31: Iteration: 198000, Train Loss: -7.0020


Training: 100%|█████████▉| 199005/200001 [1:50:34<00:31, 31.43iter/s, Iteration=199000, Train Loss=-12.5064, LR=0.000001]

2025-04-26 17:38:05: Iteration: 199000, Train Loss: -12.5064


Training: 100%|██████████| 200001/200001 [1:51:07<00:00, 29.99iter/s, Iteration=2e+5, Train Loss=-9.8728, LR=0.000001]

2025-04-26 17:38:38: Iteration: 200000, Train Loss: -9.8728
2025-04-26 17:38:38: Iteration: 200000, Test Loss: -9.8728
True Polygon:  Polygon(n=4, vertices=[(4.57, 7.97), (2.03, 5.44), (2.17, 4.0), (7.21, 2.97)], lengths=[3.5850383540486703, 1.4467895493125464, 5.144171459039833, 5.654166605256693], angles=[72.9470632210515, 129.33401968646254, 107.10318176857118, 50.61573532391479])
Predicted Polygon Vertices:  Polygon(n=4, vertices=[(4.57, 7.97), (2.03, 5.44), (2.17, 4.0), (7.21, 2.97)], lengths=[3.5850383540486703, 1.4467895493125464, 5.144171459039833, 5.654166605256693], angles=[70.17955780029297, 105.91519927978516, 103.60581970214844, 69.02519226074219])
